1. Configuración inicial: preprocesamiento
Asegúrate de que los datos estén preparados antes de entrenar el modelo:
Aplica One-Hot Encoding para las columnas categóricas.
Escala las variables numéricas si hay diferencias significativas en magnitudes.

Por ejemplo:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Dividir las variables predictoras (X) y la variable objetivo (y)
X = df.drop(columns=['price'])  # price es la variable que queremos predecir
y = df['price']  # Variable objetivo

# Codificar las columnas categóricas
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[['sex', 'smoker', 'region']])
X = pd.concat([pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out()), X.drop(columns=['sex', 'smoker', 'region'])], axis=1)

# Escalar variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

2. Entrenar el modelo de regresión lineal
Usa  para crear y entrenar el modelo:

In [ ]:
from sklearn.linear_model import LinearRegression

# Instanciar el modelo
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

3. Evaluar el modelo
Evalúa el rendimiento utilizando métricas como el Error Cuadrático Medio (MSE) y el R²:

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"R²: {r2}")

4. Interpretar los coeficientes
Los coeficientes del modelo de regresión lineal indican la influencia de cada variable en el precio del auto:
Esto te dará una tabla que muestra qué tan fuerte es la relación entre cada característica y el precio.

In [ ]:
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print(coefficients)

5. Ajustes
Si el rendimiento no es ideal:
Revisa la relevancia de las características y elimina aquellas que no aporten mucho valor.
Prueba transformaciones adicionales de las variables (como logaritmos para variables sesgadas).
Explora interacción entre las variables si sospechas de relaciones no lineales.